In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
# 일반
data_1 = pd.read_excel('./raw/6110000_서울특별시_07_24_02_P_관광유흥음식점업.xlsx')
# 휴게
data_2 = pd.read_excel('./raw/6110000_서울특별시_07_24_03_P_외국인전용유흥음식점업.xlsx')
# 외국인
data_3 = pd.read_excel('./raw/6110000_서울특별시_07_24_04_P_일반음식점.xlsx')
# 관광
data_4 = pd.read_excel('./raw/6110000_서울특별시_07_24_05_P_휴게음식점.xlsx')

/home/dialog/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
data = pd.concat([data_1,data_2,data_3,data_4],axis=0)

In [4]:
# 자치구 추출
data['자치구']=float('nan')
data.loc[data['소재지전체주소'].notna(),'자치구']=data.loc[data['소재지전체주소'].notna(),'소재지전체주소'].apply(lambda i : i.strip().split()[1])
data.loc[data['자치구'].isna(),'자치구']=data.loc[data['자치구'].isna(),'도로명전체주소'].apply(lambda i : i.strip().split()[1])

In [5]:
data_in_business=data.loc[data['영업상태명']=='영업/정상',['사업장명','소재지전체주소','도로명전체주소','자치구']]

In [6]:
data_not_in_business=data.loc[data['영업상태명']!='영업/정상',['사업장명','소재지전체주소','도로명전체주소','자치구']]

# 영업 중인 곳, 구로 나누기
- 중복 가능

In [7]:
서울시_자치구 = [i.strip() for i in  open('./서울시_자치구.txt','r',encoding='utf-8').readlines()]

In [9]:
statistic={}
for gu in 서울시_자치구:
    gu_data = {}
    tmp = data_in_business.loc[data_in_business['자치구']==gu,:]
    tmp.to_csv('./Seoul/%s.csv'%gu, index=False)
    data_not_in_business.loc[data_not_in_business['자치구']==gu,:].to_csv('./Seoul/%s_not_in_business.csv'%gu, index=False)
    statistic[gu]=len(tmp)

In [10]:
print(statistic)

{'강남구': 4066, '강동구': 1373, '강북구': 4347, '강서구': 8173, '관악구': 1359, '광진구': 5722, '구로구': 5294, '금천구': 3996, '노원구': 5023, '도봉구': 3050, '동대문구': 5479, '동작구': 1050, '마포구': 10369, '서대문구': 4970, '서초구': 2203, '성동구': 4953, '성북구': 5385, '송파구': 2552, '양천구': 4262, '영등포구': 2098, '용산구': 6025, '은평구': 5191, '종로구': 8866, '중구': 8671, '중랑구': 4785}


In [15]:
tmp

,사업장명,소재지전체주소,도로명전체주소,자치구
103364,소먹날 소한마리 정육식당,서울특별시 중랑구 망우동 490-31번지,서울특별시 중랑구 용마산로115길 76 (망우동),중랑구
103365,옆집떡볶이,서울특별시 중랑구 중화동 307-5번지,서울특별시 중랑구 동일로 831 (중화동),중랑구
103366,능이마을 사가정점,서울특별시 중랑구 면목동 648-31,서울특별시 중랑구 면목로39길 24 (면목동),중랑구
103367,황금커피&호프,서울특별시 중랑구 면목동 379-34 우진빌딩,"서울특별시 중랑구 면목로 235, 우진빌딩 (면목동)",중랑구
103368,닭발돌이,서울특별시 중랑구 면목동 67-3번지,서울특별시 중랑구 용마산로 377 (면목동),중랑구
...,...,...,...,...
27912,주노샌드,서울특별시 중랑구 면목동 354-1,"서울특별시 중랑구 면목로 278, 2층 (면목동)",중랑구
27913,서오릉피자 사가정점,서울특별시 중랑구 면목동 636-18,"서울특별시 중랑구 사가정로 384, 1층 (면목동)",중랑구
27914,이마트24 중화바른점,서울특별시 중랑구 중화동 23-2 1층,"서울특별시 중랑구 동일로136나길 37, 1층 (중화동)",중랑구
27915,요거 프레소,서울특별시 중랑구 면목동 192-65 하나스위트,"서울특별시 중랑구 봉우재로2길 3, 101호 (면목동, 하나스위트)",중랑구


In [16]:
tmp = pd.read_csv('./Seoul/강남구.csv')

In [18]:
tmp.index

RangeIndex(start=0, stop=4066, step=1)

In [19]:
tmp['query']=None

In [22]:
def query_generation(df):
    # nan
    if type(df['소재지전체주소'])==float:
        if type(df['도로명전체주소'])==float:
            address = ''
        else:
            address = df['도로명전체주소']
    else:
        address = df['소재지전체주소']
    if address: 
        address = ' '.join(address.split()[:3]).strip()
    query = address+' '+df['사업장명']
    return query.strip()

In [29]:
tmp.apply(query_generation, axis=1)

0                    서울특별시 강남구 세곡동 커스텀커피
1                서울특별시 강남구 신사동 송오브송즈로스터즈
2               서울특별시 강남구 역삼동 코코버블티 강남역점
3               서울특별시 강남구 수서동 KT그룹희망나눔재단
4                 서울특별시 강남구 논현동 에스프레소엔투샷
                      ...               
4061              서울특별시 강남구 수서동 씨유 수서럭키점
4062             서울특별시 강남구 대치동 G PC방 대치점
4063            서울특별시 강남구 역삼동 지에스25상록회관점
4064    서울특별시 강남구 대치동 주식회사 청담은 롯데백화점 강남점
4065    서울특별시 강남구 자곡동 픽처레스크(picturesque)
Length: 4066, dtype: object

In [26]:
tmp['도로명전체주소'].fillna('')

0            서울특별시 강남구 헌릉로569길 35, 지상1층 101호 (세곡동)
1       서울특별시 강남구 논현로163길 17, 압구정 디자인빌딩 지상1층 (신사동)
2                  서울특별시 강남구 강남대로96길 9, 지상1층 (역삼동)
3               서울특별시 강남구 광평로47길 28, KT 지상6층 (수서동)
4               서울특별시 강남구 언주로135길 32-3, 지상1층 (논현동)
                           ...                    
4061        서울특별시 강남구 광평로51길 49, 지상1층 (수서동, 주공아파트)
4062            서울특별시 강남구 삼성로 349, 우진빌딩 지하1층 (대치동)
4063          서울특별시 강남구 언주로98길 25, 청담빌딩 지상1층 (역삼동)
4064           서울특별시 강남구 도곡로 401, 롯데백화점 지상6층 (대치동)
4065              서울특별시 강남구 자곡로7길 32-2, 지상1층 (자곡동)
Name: 도로명전체주소, Length: 4066, dtype: object